In [33]:
# !pip install -q ptflops

In [34]:
#!pip install -r requirements.txt

In [35]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import jaccard_score
import numpy as np
import math
import torch.nn.functional as F
import torchvision
import time
import pandas as pd

In [36]:
# # 기본 디렉토리 설정 
# TRAIN_DIR = "/kaggle/input/2025-sw-ai/archive/train"
# VAL_DIR = "/kaggle/input/2025-sw-ai/archive/val"
# TEST_DIR = "/kaggle/input/2025-sw-ai/archive/test/images"
# OUTPUT_PATH = "/kaggle/working/submission.csv"

In [37]:
# 로컬 디렉토리 설정
TRAIN_DIR = "input/2025-csu-sw-ai-challenge/archive/train" 
VAL_DIR = "input/2025-csu-sw-ai-challenge/archive/val"
TEST_DIR = "input/2025-csu-sw-ai-challenge/archive/test/images"
OUTPUT_PATH = "working/submission.csv" 

In [38]:
class CrackDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.img_dir = os.path.join(root_dir, "images")
        self.mask_dir = os.path.join(root_dir, "masks")
        self.img_list = sorted(glob.glob(self.img_dir + "/*.jpg"))
        self.mask_list = sorted(glob.glob(self.mask_dir + "/*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img = Image.open(self.img_list[idx]).convert("L")  # grayscale
        mask = Image.open(self.mask_list[idx]).convert("L")

        img = np.array(img, dtype=np.float32) / 255.0
        mask = np.array(mask, dtype=np.float32) / 255.0
        mask = (mask > 0.5).astype(np.float32)  # binary mask

        img = torch.tensor(img).unsqueeze(0)  # (1, H, W)
        mask = torch.tensor(mask).unsqueeze(0)  # (1, H, W)

        if self.transform:
            img = self.transform(img)

        return img, mask

In [39]:
class HrSegNetB16(nn.Module):
    def __init__(self,
                 in_channels=1,  # input channel
                 base=16,  # base channel of the model, 
                 num_classes=1,  # number of classes
                 pretrained=None  # pretrained model
                 ):
        super(HrSegNetB16, self).__init__()
        self.base = base
        self.num_classes = num_classes
        self.pretrained = pretrained
        # Stage 1 and 2 constitute the stem of the model, which is mainly used to extract low-level features.
        # Meanwhile, stage1 and 2 reduce the input image to 1/2 and 1/4 of the original size respectively
        self.stage1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=base // 2, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(base // 2),
            nn.ReLU(),
        )
        self.stage2 = nn.Sequential(
            nn.Conv2d(in_channels=base // 2, out_channels=base, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(base),
            nn.ReLU(),
        )

        self.seg1 = SegBlock(base=base, stage_index=1)
        self.seg2 = SegBlock(base=base, stage_index=2)
        self.seg3 = SegBlock(base=base, stage_index=3)

        self.aux_head1 = SegHead(inplanes=base, interplanes=base, outplanes=num_classes, aux_head=True)
        self.aux_head2 = SegHead(inplanes=base, interplanes=base, outplanes=num_classes, aux_head=True)
        self.head = SegHead(inplanes=base, interplanes=base, outplanes=num_classes)

        self.init_weight()
    
    def forward(self, x):
        logit_list = []
        h, w = x.shape[2:]
        # aux_head only used in training
        if self.training:
            stem1_out = self.stage1(x)
            stem2_out = self.stage2(stem1_out)
            hrseg1_out = self.seg1(stem2_out)
            hrseg2_out = self.seg2(hrseg1_out)
            hrseg3_out = self.seg3(hrseg2_out)
            last_out = self.head(hrseg3_out)
            seghead1_out = self.aux_head1(hrseg1_out)
            seghead2_out = self.aux_head2(hrseg2_out)
            logit_list = [last_out, seghead1_out, seghead2_out]
            logit_list = [F.interpolate(logit, size=(h, w), mode='bilinear', align_corners=True) for logit in logit_list]
            return logit_list
        else:
            stem1_out = self.stage1(x)
            stem2_out = self.stage2(stem1_out)
            hrseg1_out = self.seg1(stem2_out)
            hrseg2_out = self.seg2(hrseg1_out)
            hrseg3_out = self.seg3(hrseg2_out)
            last_out = self.head(hrseg3_out)
            logit_list = [last_out]
            logit_list = [F.interpolate(logit, size=(h, w), mode='bilinear', align_corners=True) for logit in logit_list]
            return logit_list
        
    
    def init_weight(self):
        if self.pretrained is not None:
            pass
        else:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
    
class SegBlock(nn.Module):
    def __init__(self, base=32, stage_index=1):
        super(SegBlock, self).__init__()

        # Convolutional layer for high-resolution paths with constant spatial resolution and constant channel
        self.h_conv1 = nn.Sequential(
            nn.Conv2d(in_channels=base, out_channels=base, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(base),
            nn.ReLU()
        )
        self.h_conv2 = nn.Sequential(
            nn.Conv2d(in_channels=base, out_channels=base, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(base),
            nn.ReLU()
        )
        self.h_conv3 = nn.Sequential(
            nn.Conv2d(in_channels=base, out_channels=base, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(base),
            nn.ReLU()
        )

        # Semantic guidance path/low-resolution path
        if stage_index == 1:  # First stage, stride=2, spatial resolution/2, channel*2
            self.l_conv1 = nn.Sequential(
                nn.Conv2d(in_channels=base, out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
                nn.ReLU()
            )
        elif stage_index == 2:  # Second stage
            self.l_conv1 = nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                nn.Conv2d(in_channels=base, out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
                nn.ReLU()
            )
        elif stage_index == 3:
            self.l_conv1 = nn.Sequential(
                nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
                nn.Conv2d(in_channels=base, out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
                nn.ReLU(),
                nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
                nn.ReLU()
            )
        else:
            raise ValueError("stage_index must be 1, 2 or 3")
        self.l_conv2 = nn.Sequential(
            nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
            nn.ReLU()
        )
        self.l_conv3 = nn.Sequential(
            nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base * int(math.pow(2, stage_index)), kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(base * int(math.pow(2, stage_index))),
            nn.ReLU()
        )

        self.l2h_conv1 = nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base, kernel_size=1, stride=1, padding=0)
        self.l2h_conv2 = nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base, kernel_size=1, stride=1, padding=0)
        self.l2h_conv3 = nn.Conv2d(in_channels=base * int(math.pow(2, stage_index)), out_channels=base, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        size = x.shape[2:]
        out_h1 = self.h_conv1(x)  # high resolution path
        out_l1 = self.l_conv1(x)  # low resolution path
        out_l1_i = F.interpolate(out_l1, size=size, mode='bilinear', align_corners=True)  # upsample
        out_hl1 = self.l2h_conv1(out_l1_i) + out_h1  # low to high

        out_h2 = self.h_conv2(out_hl1)
        out_l2 = self.l_conv2(out_l1)
        out_l2_i = F.interpolate(out_l2, size=size, mode='bilinear', align_corners=True)
        out_hl2 = self.l2h_conv2(out_l2_i) + out_h2

        out_h3 = self.h_conv3(out_hl2)
        out_l3 = self.l_conv3(out_l2)
        out_l3_i = F.interpolate(out_l3, size=size, mode='bilinear', align_corners=True)
        out_hl3 = self.l2h_conv3(out_l3_i) + out_h3
        return out_hl3

# seg head
class SegHead(nn.Module):
    def __init__(self, inplanes, interplanes, outplanes, aux_head=False):
        super(SegHead, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU()
        if aux_head:
            self.con_bn_relu = nn.Sequential(
                nn.Conv2d(in_channels=inplanes, out_channels=interplanes, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(interplanes),
                nn.ReLU(),
            )
        else:
            self.con_bn_relu = nn.Sequential(
                nn.ConvTranspose2d(in_channels=inplanes, out_channels=interplanes, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm2d(interplanes),
                nn.ReLU(),
            )
        self.conv = nn.Conv2d(in_channels=interplanes, out_channels=outplanes, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.bn1(x)
        x = self.relu(x)
        x = self.con_bn_relu(x)
        out = self.conv(x)
        return out

In [40]:
def binary_metrics(preds, targets, eps=1e-6):
    preds = preds.float()
    targets = targets.float()

    tp = (preds * targets).sum(dim=(1,2,3))
    fp = (preds * (1 - targets)).sum(dim=(1,2,3))
    fn = ((1 - preds) * targets).sum(dim=(1,2,3))

    precision = (tp + eps) / (tp + fp + eps)
    recall    = (tp + eps) / (tp + fn + eps)
    f1        = (2 * precision * recall + eps) / (precision + recall + eps)  # Dice
    union     = tp + fp + fn
    iou       = (tp + eps) / (union + eps)

    return {
        "iou": iou.mean().item(),
        "precision": precision.mean().item(),
        "recall": recall.mean().item(),
        "f1": f1.mean().item(),
    }

def train_model(
    model,
    train_loader,
    val_loader,
    device,
    epochs=10,
    aux_weights=(1.0, 0.4, 0.4),
    lr=1e-3,
    use_amp=False,
    log_every=500,
    validate_every_steps=None,
    threshold=0.5,):   
    
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    global_step = 0
    win_loss, win_iou, win_f1, win_steps = 0.0, 0.0, 0.0, 0
    t0 = time.time()

    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0

        for imgs, masks in train_loader:
            global_step += 1
            imgs = imgs.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast(enabled=use_amp):
                outputs = model(imgs)  # train: [main, aux1, aux2]
                main_logit = outputs[0] if isinstance(outputs, (list, tuple)) else outputs
                loss = aux_weights[0] * criterion(main_logit, masks)
                if isinstance(outputs, (list, tuple)):
                    if len(outputs) > 1 and aux_weights[1] > 0:
                        loss = loss + aux_weights[1] * criterion(outputs[1], masks)
                    if len(outputs) > 2 and aux_weights[2] > 0:
                        loss = loss + aux_weights[2] * criterion(outputs[2], masks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # 집계
            epoch_loss += loss.item()
            win_loss += loss.item()
            win_steps += 1

            with torch.no_grad():
                probs = torch.sigmoid(main_logit)
                preds = (probs > threshold).float()
                m = binary_metrics(preds, masks)
                win_iou += m["iou"]
                win_f1  += m["f1"]

            if log_every and (global_step % log_every == 0):
                elapsed = time.time() - t0
                lr_now = optimizer.param_groups[0]["lr"]
                print(f"[Step {global_step}] epoch={epoch}  "
                      f"avg_loss(win)={win_loss/max(1,win_steps):.4f}  "
                      f"avg_iou(win)={win_iou/max(1,win_steps):.4f}  "
                      f"avg_f1(win)={win_f1/max(1,win_steps):.4f}  "
                      f"lr={lr_now:.3e}  elapsed={elapsed:.1f}s")
                win_loss = win_iou = win_f1 = 0.0
                win_steps = 0
                t0 = time.time()

            if validate_every_steps and (global_step % validate_every_steps == 0):
                model.eval()
                val_iou_list, val_f1_list = [], []
                with torch.no_grad():
                    for v_imgs, v_masks in val_loader:
                        v_imgs = v_imgs.to(device, non_blocking=True)
                        v_masks = v_masks.to(device, non_blocking=True)
                        out_list = model(v_imgs)   # eval: [main]만
                        main_logit_eval = out_list[0] if isinstance(out_list, (list, tuple)) else out_list
                        preds = (torch.sigmoid(main_logit_eval) > threshold).float()
                        m = binary_metrics(preds, v_masks)
                        val_iou_list.append(m["iou"])
                        val_f1_list.append(m["f1"])
                print(f"[Step {global_step}] 🔍 Val IoU={np.mean(val_iou_list):.4f} | Val F1={np.mean(val_f1_list):.4f}")
                model.train()

        avg_train_loss = epoch_loss / max(1, len(train_loader))
        model.eval()
        val_iou_list, val_f1_list = [], []
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs = imgs.to(device, non_blocking=True)
                masks = masks.to(device, non_blocking=True)
                logits_list = model(imgs)  # eval: [main]
                main_logit = logits_list[0] if isinstance(logits_list, (list, tuple)) else logits_list
                preds = (torch.sigmoid(main_logit) > threshold).float()
                m = binary_metrics(preds, masks)
                val_iou_list.append(m["iou"])
                val_f1_list.append(m["f1"])
        print(f"[Epoch {epoch}/{epochs}] "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val IoU: {np.mean(val_iou_list):.4f} | "
              f"Val F1: {np.mean(val_f1_list):.4f}")


In [41]:
train_dataset = CrackDataset(TRAIN_DIR)
val_dataset = CrackDataset(VAL_DIR)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [42]:
from ptflops import get_model_complexity_info

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HrSegNetB16().to(device)
input_size = (1, 192, 192)

macs, params = get_model_complexity_info(model,
                                         input_size,
                                         as_strings=True,
                                         print_per_layer_stat=False,
                                         verbose=False)
print(f"Total Params: {params}")
print(f"Total MACs: {macs}")

Total Params: 609.75 k
Total MACs: 130.86 MMac


In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = HrSegNetB16().to(device)
train_model(model, train_loader, val_loader, device, epochs=20)

cuda


/tmp/ipykernel_40006/3727572993.py:38: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/tmp/ipykernel_40006/3727572993.py:55: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[Step 500] epoch=1  avg_loss(win)=0.4380  avg_iou(win)=0.1449  avg_f1(win)=0.2039  lr=1.000e-03  elapsed=11.9s
[Epoch 1/20] Train Loss: 0.3325 | Val IoU: 0.2190 | Val F1: 0.2992
[Step 1000] epoch=2  avg_loss(win)=0.2131  avg_iou(win)=0.2169  avg_f1(win)=0.3190  lr=1.000e-03  elapsed=13.3s
[Step 1500] epoch=2  avg_loss(win)=0.1799  avg_iou(win)=0.2699  avg_f1(win)=0.3956  lr=1.000e-03  elapsed=11.5s
[Epoch 2/20] Train Loss: 0.1765 | Val IoU: 0.3537 | Val F1: 0.4636
[Step 2000] epoch=3  avg_loss(win)=0.1650  avg_iou(win)=0.3298  avg_f1(win)=0.4610  lr=1.000e-03  elapsed=12.6s
[Step 2500] epoch=3  avg_loss(win)=0.1599  avg_iou(win)=0.3619  avg_f1(win)=0.4914  lr=1.000e-03  elapsed=11.9s
[Epoch 3/20] Train Loss: 0.1569 | Val IoU: 0.4195 | Val F1: 0.5370
[Step 3000] epoch=4  avg_loss(win)=0.1489  avg_iou(win)=0.3838  avg_f1(win)=0.5094  lr=1.000e-03  elapsed=13.7s
[Step 3500] epoch=4  avg_loss(win)=0.1497  avg_iou(win)=0.3914  avg_f1(win)=0.5146  lr=1.000e-03  elapsed=11.1s
[Epoch 4/20] Tra

In [44]:

def rle_encode(mask):
    """
    mask: 2D numpy array of {0,1}, shape (H,W)
    return: run length as string
    """
    pixels = mask.flatten(order="C")
    ones = np.where(pixels == 1)[0] + 1  # 1-based
    if len(ones) == 0:
        return ""
    runs = []
    prev = -2
    for idx in ones:
        if idx > prev + 1:
            runs.extend((idx, 0))
        runs[-1] += 1
        prev = idx
    return " ".join(map(str, runs))


def predict_and_submit(model, test_img_dir, output_csv, device, threshold=0.5):
    model.eval()
    ids, rles = [], []

    test_imgs = sorted(glob.glob(os.path.join(test_img_dir, "*.jpg")))
    for path in test_imgs:
        img_id = os.path.splitext(os.path.basename(path))[0]
        img = Image.open(path).convert("L")
        arr = np.array(img, dtype=np.float32) / 255.0
        tensor = torch.tensor(arr).unsqueeze(0).unsqueeze(0).to(device)

        with torch.no_grad():
            out_list = model(tensor)
            main_logit = out_list[0] if isinstance(out_list, (list, tuple)) else out_list
            prob = torch.sigmoid(main_logit)[0,0].cpu().numpy()
            pred = (prob > threshold).astype(np.uint8)

        rle = rle_encode(pred)
        ids.append(img_id)
        rles.append(rle)

    df = pd.DataFrame({"image_id": ids, "rle": rles})
    df.to_csv(output_csv, index=False)
    print(f"[OK] submission saved to {output_csv}, total {len(df)} rows.")


In [45]:
predict_and_submit(
    model, 
    test_img_dir=TEST_DIR,
    output_csv=OUTPUT_PATH,
    device=device,
    threshold=0.5
)

[OK] submission saved to working/submission.csv, total 2667 rows.
